In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import keras.layers as layers
import keras.optimizers as optimizers
from keras.models import Model, load_model
from keras.utils import to_categorical
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau
import seaborn as sns
from PIL import Image
from skimage.transform import resize

import threading, random, os

# Data Generator

원본 이미지와 라벨 데이터를 가져오고, 이 이미지와 라벨을 패널로 나누어 학습 데이터와 라벨을 생성하는 커스텀 생성기를 작성하는 부분.

## Load Dataset

원본 이미지와 라벨 데이터를 가져온다. 코드를 실행하기 전에 코드와 같은 폴더에 데이터셋을 다운받는다.   

https://www.kaggle.com/kairess/find-waldo

In [2]:
imgs = np.load('dataset/imgs_uint8.npy').astype(np.float32) / 255.
labels = np.load('dataset/labels_uint8.npy').astype(np.float32) / 255.
waldo_sub_imgs = np.load('dataset/waldo_sub_imgs_uint8.npy') / 255.
waldo_sub_labels = np.load('dataset/waldo_sub_labels_uint8.npy') / 255.

ValueError: Object arrays cannot be loaded when allow_pickle=False

원본 코드 작성시와 numpy 버전 차이가 있어 waldo_sub_imgs와 waldo_sub_labels를 로드할 때 오류가 발생한다. allow_pickle 파라메터의 기본값이 True에서 False로 바뀌어 발생하는 문제로 ```allow_pickle=True```를 코드에 추가해주면 해결된다.

pickle은 파이썬에서 제공하는 모듈로 자료형의 변화 없이 데이터를 저장하고 불러올 수 있게해준다. 다만 악의적으로 제작된 데이터를 불러올 경우 임의로 코드를 실행하는 문제가 있을 수 있기 때문에 보안상의 문제로 기본값이 변경된 것으로 보인다.

In [3]:
imgs = np.load('dataset/imgs_uint8.npy').astype(np.float32) / 255.
labels = np.load('dataset/labels_uint8.npy').astype(np.float32) / 255.
waldo_sub_imgs = np.load('dataset/waldo_sub_imgs_uint8.npy', allow_pickle=True) / 255.
waldo_sub_labels = np.load('dataset/waldo_sub_labels_uint8.npy', allow_pickle=True) / 255.

In [4]:
print(imgs.shape, labels.shape)

(18, 1760, 2800, 3) (18, 1760, 2800)


먼저 imgs와 labels의 형태를 확인한다.   
imgs는 1760*2800 크기의 이미지 18장으로 RGB 데이터값이 입력된 numpy배열이다.
labels는 똑같은 이미지에서 waldo가 있는 부분을 255, 아닌 부분을 0으로 표시한 numpy배열이다.

데이터를 학습에 적합한 형태로 (0과 1사이의 값으로) 바꾸기 위해 255로 나눠준다.


In [6]:
print(waldo_sub_imgs.shape, waldo_sub_labels.shape)
print(waldo_sub_imgs[0].shape, waldo_sub_imgs[1].shape)

(18,) (18,)
(379, 397, 3) (386, 394, 3)


다음으로 waldo_sub_imgs와 waldo_sub_labels의 형태를 확인한다.
waldo_sub_imgs는 imgs에서 waldo가 있는 부분과 그 주변을 잘라낸 이미지로 imgs와 마찬가지로 RGB 데이터값을 가진 이미지 18장의 numpy배열이다.   
자른 크기는 각각 다름을 확인할 수 있다.
waldo_sub_labels는 labels와 동일하게 waldo_sub_imgs와 동일한 크기인 waldo가 있는 부분을 255, 아닌 부분을 0으로 표시한 numpy 배열이다.

역시 데이터를 학습에 적합한 형태로 바꾸기 위해 255로 나눠준다.

## Data Generator

이 코드에서는 이미지를 패널로 나누어 모델 학습을 진행한다.   
학습을 위해 이미지와 라벨을 패널로 분할해 데이터를 만드는 생성기를 작성한다.

In [8]:
class BatchIndices(object):
    def __init__(self, n, bs, shuffle=False):
        self.n,self.bs,self.shuffle = n,bs,shuffle
        self.lock = threading.Lock()
        self.reset()

    def reset(self):
        self.idxs = (np.random.permutation(self.n) 
                     if self.shuffle else np.arange(0, self.n))
        self.curr = 0

    def __next__(self):
        with self.lock:
            if self.curr >= self.n: self.reset()
            ni = min(self.bs, self.n-self.curr)
            res = self.idxs[self.curr:self.curr+ni]
            self.curr += ni
            return res
        
class segm_generator(object):
    def __init__(self, x, y, bs=64, out_sz=(224,224), train=True, waldo=True):
        self.x, self.y, self.bs, self.train = x,y,bs,train
        self.waldo = waldo
        self.n = x.shape[0]
        self.ri, self.ci = [], []
        for i in range(self.n):
            ri, ci, _ = x[i].shape
            self.ri.append(ri), self.ci.append(ci) 
        self.idx_gen = BatchIndices(self.n, bs, train)
        self.ro, self.co = out_sz
        
    def get_slice(self, i,o):
        start = random.randint(0, i-o) if self.train else (i-o)
        return slice(start, start+o)

    def get_item(self, idx):
        slice_r = self.get_slice(self.ri[idx], self.ro)
        slice_c = self.get_slice(self.ci[idx], self.co)
        x = self.x[idx][slice_r, slice_c]
        y = self.y[idx][slice_r, slice_c]
        if self.train and (random.random()>0.5): 
            y = y[:,::-1]
            x = x[:,::-1]
        if not self.waldo and np.sum(y)!=0:
            return None

        return x, to_categorical(y, num_classes=2).reshape((y.shape[0] * y.shape[1], 2))

    def __next__(self):
        idxs = self.idx_gen.__next__()
        items = []
        for idx in idxs:
            item = self.get_item(idx)
            if item is not None:
                items.append(item)
        if not items:
            return None
        xs,ys = zip(*tuple(items))
        return np.stack(xs), np.stack(ys)
        
def seg_gen_mix(x1, y1, x2, y2, tot_bs=4, prop=0.34, out_sz=(224,224), train=True):
    n1 = int(tot_bs*prop)
    n2 = tot_bs - n1
    sg1 = segm_generator(x1, y1, n1, out_sz = out_sz ,train=train)
    sg2 = segm_generator(x2, y2, n2, out_sz = out_sz ,train=train, waldo=False)
    while True:
        out1 = sg1.__next__()
        out2 = sg2.__next__()
        if out2 is None:
            yield out1
        else:
            yield np.concatenate((out1[0], out2[0])), np.concatenate((out1[1], out2[1]))            

seg_gen_mix라는 함수를 만들기 위해 BatchIndice와 segm_generator라는 클래스를 설정한다. 각각의 변수가 어떤 의미를 가지는지 이해하는 것이 중요하다.

변수 확인을 위해 seg_gen_mix 함수의 파라메터를 먼저 확인한다.   
x1, y1은 위에서 가져온 waldo_sub_imgs와 waldo_sub_labels, x2, y2는 imgs와 labels가 들어간다.   
tot_bs와 prop는 한번의 batch에서 생성할 패널의 최대 개수와 그 중 x1과 y1을 이용해 생성할 패널의 비율을 설정하는 파라메터이다.   
out_sz는 생성할 패널의 크기를 지정한다.   
train은 추가적인 옵션을 바꾸기 위해 설정한 boolean값이다.

이제 함수에서 사용되는 segm_generator instance 생성 부분 파라메터값을 보면 이미지와 라벨, tot_bs와 prop를 활용하여 만든 생성할 이미지의 개수, 패널의 크기, train 옵션, waldo 옵션이 들어가는걸 볼 수 있다. imgs와 labels를 이용한 instance 생성에는 waldo가 False로 설정되어있다.

segm_generator class를 설정하는 부분에서 BatchIndice instance를 생성하는 부분을 보면 self.n, bs, train 값을 전달하는걸 확인할 수 있다. bs와 train은 앞서 전달받은 생성할 이미지의 개수와 train 옵션의 값이고 self.n의 경우 들어온 이미지의 0번 인덱스 즉 입력된 데이터의 이미지 개수(이 경우에는 18장)이다.

In [ ]:
'''
class BatchIndices(object):
    def __init__(self, n, bs, shuffle=False):
        self.n,self.bs,self.shuffle = n,bs,shuffle
        self.lock = threading.Lock()
        self.reset()

    def reset(self):
        self.idxs = (np.random.permutation(self.n) 
                     if self.shuffle else np.arange(0, self.n))
        self.curr = 0

    def __next__(self):
        with self.lock:
            if self.curr >= self.n: self.reset()
            ni = min(self.bs, self.n-self.curr)
            res = self.idxs[self.curr:self.curr+ni]
            self.curr += ni
            return res
'''